<a href="https://colab.research.google.com/github/Lean-IQ/Stockmarket-Analysis/blob/main/Stockmarket_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stock Market Analysis

**About**

This Python script is designed for analyzing stock market performance using a selection of technical indicators. The primary goals of this script are:

1.   Trend Analysis: Identify and analyze market trends using indicators such as Moving Averages (MA), Relative Strength Index (RSI), and Moving Average Convergence Divergence (MACD).

2.   Signal Generation: Generate buy and sell signals based on the computed indicators, helping to inform trading decisions.

3.   Performance Evaluation: Assess the performance of selected stocks over time, providing insights into potential investment opportunities.

4.   Visualization: Provide intuitive visual representations of stock data and indicators, aiding in the interpretation of market trends and signals.

5. Data-Driven Insights: Utilize historical stock data to backtest strategies and validate the effectiveness of selected indicators.

6. Automation: Automate the process of data retrieval, analysis, and visualization to ensure efficiency and accuracy.

7. Customization: Allow users to customize the parameters of technical indicators to tailor the analysis to specific trading strategies or preferences.

This script leverages Python libraries such as pandas for data manipulation, matplotlib for plotting, and yfinance for retrieving stock data, ensuring a comprehensive and efficient analysis toolkit for market participants.

**License**

This project is licensed under the MIT License. You are free to use, modify, and distribute this software in accordance with the terms specified in the MIT License. A copy of the license is included in the project repository.

##Loading Required Libraries

In [1]:
pip install yfinance

## Raw Data Picking

**Usage**

* Dependencies: Ensure you have the required Python libraries installed, including yfinance, pandas, and google.colab (if running in Google Colab).

* Google Drive Setup: If running in Google Colab, mount your Google Drive to access files using the appropriate commands.

* File Paths: Define the paths to your CSV file containing ticker symbols and the directory where raw data will be saved. Modify the csv_file_path and raw_data_path variables as needed to match your directory structure.

* Running the Script: Execute the script to fetch and save data for each ticker symbol in your portfolio. Load the CSV file containing ticker symbols and iterate over each symbol to fetch and save historical data.

**Script Explanation**

The script performs the following key tasks:

* Mount Google Drive: Mounts Google Drive to access the necessary files if running in Google Colab.

* Define File Paths: Specifies the paths to the CSV file with ticker symbols and the directory for saving raw data.

* Check File Age: Contains a function to check if a file exists and is older than one day, ensuring data is up-to-date.

* Fetch and Save Data: Defines a function to fetch historical stock data for a given ticker symbol using the yfinance library, and saves the data to a CSV file with specific formatting. The function also handles errors and prints relevant messages.

* Load Ticker Symbols: Loads ticker symbols from the specified CSV file into a DataFrame.

* Iterate and Fetch Data: Iterates over the ticker symbols in the DataFrame, calling the function to fetch and save data for each symbol.

In [2]:
# PICK RAW FINANCIAL AND STOCK DATA FOR ALL PORTFOLIO COMPANIES

import os
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# Mount Google Drive if necessary
from google.colab import drive
drive.mount('/content/drive')

# Define file paths
csv_file_path = '/content/drive/MyDrive/DATA/portfolio.csv'
raw_data_path = '/content/drive/MyDrive/DATA/RAW/'

# Function to check if a file exists and is older than 1 day
def is_file_older_than_1_day(file_path):
    if not os.path.exists(file_path):
        return False
    modification_time = os.path.getmtime(file_path)
    time_difference = datetime.now() - datetime.fromtimestamp(modification_time)
    return time_difference.days > 1

# Function to fetch historical data for a ticker symbol and save it to CSV
def fetch_and_save_data(ticker):
    try:
        # Check if CSV file already exists and is older than 1 day
        ticker_csv_path = os.path.join(raw_data_path, f'{ticker}.csv')
        if is_file_older_than_1_day(ticker_csv_path):
            os.remove(ticker_csv_path)  # Remove the file if it's older than 1 day

        # Fetch last trading day
        last_trading_day = yf.download("SPY").index[-1]

        # Calculate start date 1 year back from the last trading day
        start_date = last_trading_day - timedelta(days=365)

        # Fetch historical data
        ticker_data = yf.download(ticker, start=start_date, end=last_trading_day)

        # Save data to CSV with specific formatting
        ticker_data.to_csv(ticker_csv_path, float_format='%.5f')  # Control decimal precision
        print(f"Data for {ticker} saved to {ticker_csv_path}")
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

# Load CSV file containing ticker symbols
portfolio_df = pd.read_csv(csv_file_path)

# Iterate over ticker symbols and fetch data
for ticker in portfolio_df['YFINANCE']:
    fetch_and_save_data(ticker)

Mounted at /content/drive


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Data for ^DJI saved to /content/drive/MyDrive/DATA/RAW/^DJI.csv


[*********************100%%**********************]  1 of 1 completed


Data for ^GDAXI saved to /content/drive/MyDrive/DATA/RAW/^GDAXI.csv


...

Data for XEL saved to /content/drive/MyDrive/DATA/RAW/XEL.csv


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Data for XYL saved to /content/drive/MyDrive/DATA/RAW/XYL.csv


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Data for YAR.OL saved to /content/drive/MyDrive/DATA/RAW/YAR.OL.csv


## Calculating Stochastic Indicator

**Usage**

- Dependencies: Ensure you have the required Python libraries installed, including pandas and numpy.

- File Paths: Define the paths for the raw data directory and the output directory. Modify the raw_data_path and output_path variables to match your directory structure.

- Data Preparation: Ensure your raw data CSV files are placed in the specified raw data directory. The script expects the files to have columns such as 'Low', 'High', and 'Close'.

- Running the Script: Execute the script to analyze the Stochastic indicator for each CSV file in the raw data directory and save the results to the output directory.

**Script Explanation**

The script performs the following key tasks:

- File Paths: Specifies the paths for the raw data directory and the directory where the analyzed data will be saved.

- Stochastic Calculation: Defines a function to calculate the Stochastic indicator, including the %K and %D lines, based on given periods.

- Stochastic Analysis: Defines a function to analyze the Stochastic indicator for various trading signals and divergences. The function identifies trends such as bullish and bearish divergences, crossover signals, and potential buy/sell signals.

- Analyze Single CSV: Contains a function to load a single CSV file, calculate the Exponential Moving Average (EMA200), apply the Stochastic calculation and analysis, and save the results to the output directory. The function ensures the output is saved with appropriate formatting and decimal separators.

- Analyze All CSV Files: Defines a function to iterate over all CSV files in the raw data directory and apply the analysis function to each file. It collects the list of CSV files, processes each file, and prints a completion message once all files are analyzed.

- Execution: Calls the function to analyze all CSV files in the raw data directory, triggering the Stochastic calculation and analysis for each file, and saving the results to the specified output directory.

In [3]:
# 02STOCH

import os
import pandas as pd
import numpy as np

# Define file paths
raw_data_path = '/content/drive/MyDrive/DATA/RAW'
output_path = '/content/drive/MyDrive/DATA/STOCHASTIC'

# Function to calculate Stochastic indicator
def calculate_stochastic(df, k_period=14, d_period=3):
    # Calculate %K
    lowest_low = df['Low'].rolling(window=k_period).min()
    highest_high = df['High'].rolling(window=k_period).max()
    df['%K'] = 100 * (df['Close'] - lowest_low) / (highest_high - lowest_low)
    # Calculate %D
    df['%D'] = df['%K'].rolling(window=d_period).mean()
    return df

# Function to analyze Stochastic indicator
def analyze_stochastic(df):
    # Trending Bullish Divergence
    df['Trending Bullish Divergence'] = np.where((df['Close'] > df['EMA200']) & (df['%K'] < df['%D']), 'Yes', 'No')
    # Regular Bullish Divergence
    df['Regular Bullish Divergence'] = np.where((df['Close'].shift(1) < df['Close']) & (df['%K'].shift(1) > df['%K']) & (df['%D'].shift(1) < df['%D']), 'Yes', 'No')
    # Hidden Bullish Divergence
    df['Hidden Bullish Divergence'] = np.where((df['Close'].shift(1) > df['Close']) & (df['%K'].shift(1) < df['%K']) & (df['%D'].shift(1) > df['%D']), 'Yes', 'No')
    # Trending Bearish Divergence
    df['Trending Bearish Divergence'] = np.where((df['Close'] < df['EMA200']) & (df['%K'] > df['%D']), 'Yes', 'No')
    # Regular Bearish Divergence
    df['Regular Bearish Divergence'] = np.where((df['Close'].shift(1) > df['Close']) & (df['%K'].shift(1) < df['%K']) & (df['%D'].shift(1) > df['%D']), 'Yes', 'No')
    # Hidden Bearish Divergence
    df['Hidden Bearish Divergence'] = np.where((df['Close'].shift(1) < df['Close']) & (df['%K'].shift(1) > df['%K']) & (df['%D'].shift(1) < df['%D']), 'Yes', 'No')
    # Middle Filter Aktive
    df['Middle Filter Aktive'] = np.where((df['%K'] > 50) & (df['%D'] > 50), 'Yes', 'No')
    # Short Signal
    df['Short Signal'] = np.where((df['Close'] < df['EMA200']) & (df['%K'] > 80) & (df['%K'].shift(1) > 80), 'Yes', 'No')
    # Long Signal
    df['Long Signal'] = np.where((df['Close'] > df['EMA200']) & (df['%K'] < 20) & (df['%K'].shift(1) < 20), 'Yes', 'No')
    # Crossing
    df['Sell Signal'] = np.where((df['%K'] < df['%D']) & (df['%K'].shift(1) > df['%D'].shift(1)), 'Yes', 'No')
    df['Sell Signal, Extreme'] = np.where((df['%K'] < 80) & (df['%D'] < 80) & (df['%K'].shift(1) > 80) & (df['%D'].shift(1) > 80), 'Yes', 'No')
    df['Buy Signal'] = np.where((df['%K'] > df['%D']) & (df['%K'].shift(1) < df['%D'].shift(1)), 'Yes', 'No')
    df['Buy Signal, Extreme'] = np.where((df['%K'] > 20) & (df['%D'] > 20) & (df['%K'].shift(1) < 20) & (df['%D'].shift(1) < 20), 'Yes', 'No')
    # Crossover
    df['Right Hand Crossover'] = np.where((df['%D'].shift(1) > df['%K'].shift(1)) & (df['%D'] < df['%K']), 'Yes', 'No')
    df['Left Hand Crossover'] = np.where((df['%K'].shift(1) > df['%D'].shift(1)) & (df['%K'] < df['%D']), 'Yes', 'No')
    return df

# Function to analyze Stochastic for a single CSV file
def analyze_single_csv(file_path):
    try:
        # Load data
        df = pd.read_csv(file_path)
        # Calculate EMA200
        df['EMA200'] = df['Close'].ewm(span=200, min_periods=0).mean()
        # Calculate Stochastic indicator
        df = calculate_stochastic(df)
        # Analyze Stochastic indicator
        df = analyze_stochastic(df)
        # Output results
        filename = os.path.basename(file_path)
        output_file_path = os.path.join(output_path, filename)
        df.to_csv(output_file_path, index=False, mode='w', decimal=',')  # Ensure overwrite mode and use "," as decimal separator
    except Exception as e:
        print(f"Error analyzing Stochastic for {file_path}: {e}")

# Function to analyze Stochastic for all CSV files in the RAW folder
def analyze_all_csv_files():
    # Get list of CSV files
    csv_files = [os.path.join(raw_data_path, file) for file in os.listdir(raw_data_path) if file.endswith('.csv')]
    # Analyze Stochastic for each CSV file
    for file_path in csv_files:
        analyze_single_csv(file_path)
    print("Stochastic analysis for all files completed.")

# Analyze Stochastic for all CSV files in the RAW folder
analyze_all_csv_files()

Stochastic analysis for all files completed.


**Stochastics Data Consolidation**

In [4]:
# 03SUMSTO

import os
import pandas as pd

# Define paths
stochastic_path = '/content/drive/MyDrive/DATA/STOCHASTIC'
output_file_path = '/content/drive/MyDrive/DATA/results_stochastic.csv'

# Function to read CSV files and extract rows with the latest date
def extract_latest_data(file_path, file_name):
    try:
        df = pd.read_csv(file_path)
        latest_date = df['Date'].max()  # Find the latest date in the dataframe
        latest_data = df[df['Date'] == latest_date].copy()  # Extract rows with the latest date and create a copy
        latest_data['File'] = file_name  # Add a new column with the file name
        return latest_data
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None

# Initialize an empty list to store dataframes
dfs = []

# Iterate through CSV files in the STOCHASTIC input path
for file_name in os.listdir(stochastic_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(stochastic_path, file_name)
        # Extract rows with the latest date from the current file
        latest_data = extract_latest_data(file_path, file_name)
        if latest_data is not None:
            # Append the latest data to the list of dataframes
            dfs.append(latest_data)

# Check if any dataframes were extracted
if dfs:
    # Concatenate dataframes in the list into a single dataframe
    consolidated_data = pd.concat(dfs, ignore_index=True)

    # Check if the output file already exists and remove it
    if os.path.exists(output_file_path):
        os.remove(output_file_path)

    # Save the consolidated data to a CSV file
    consolidated_data.to_csv(output_file_path, index=False, decimal=',')
    print("Latest STOCHASTIC data with file names has been consolidated and saved to:", output_file_path)
else:
    print("No STOCHASTIC data found in the specified directory.")

Latest STOCHASTIC data with file names has been consolidated and saved to: /content/drive/MyDrive/DATA/results_stochastic.csv


##Calculating Relative Strength Indicator (RSI)

**Usage**

- Dependencies: Ensure you have the required Python libraries installed, including pandas and numpy.

- File Paths: Define the paths for the raw data directory and the RSI output directory. Modify the raw_data_path and rsi_output_path variables to match your directory structure.

- Data Preparation: Ensure your raw data CSV files are placed in the specified raw data directory. The script expects the files to have a 'Close' column for the calculation of RSI.

- Running the Script: Execute the script to calculate and interpret the RSI for each CSV file in the raw data directory and save the results to the RSI output directory.

**Script Explanation**

The script performs the following key tasks:

- File Paths: Specifies the paths for the raw data directory and the directory where the RSI data will be saved.

- RSI Calculation: Defines a function to calculate the Relative Strength Index (RSI) over a given period. This function calculates the RSI based on the difference in closing prices over time, distinguishing between gains and losses.

- RSI Signal Classification: Defines a function to classify RSI values into signals such as "Overbought", "Oversold", and "Neutral" based on standard RSI thresholds.

- RSI Interpretation: Contains a function to interpret RSI signals, including:

1. *Calculation of 14-day and 30-day RSI values.*

2. *Classification of RSI signals.*

3. *Trend interpretation based on RSI changes.*

4. *Divergence interpretation using moving averages for detecting bullish and bearish divergences.*

- Read and Process Data: Iterates over all CSV files in the raw data directory, reads each file into a DataFrame, and applies the RSI interpretation function. The interpreted data includes RSI values, signal classifications, trend directions, and divergence indicators.*

- Save Results: Saves the interpreted RSI data for each processed CSV file to the specified RSI output directory, ensuring the results are stored with appropriate formatting.

- Execution: After processing all CSV files, the script prints a completion message indicating that the RSI calculation and interpretation are finished and the results are saved.

In [5]:
# 04RSI

import os
import pandas as pd
import numpy as np

# Define paths
raw_data_path = '/content/drive/MyDrive/DATA/RAW'
rsi_output_path = '/content/drive/MyDrive/DATA/RSI'

# Function to calculate RSI
def calculate_rsi(data, period):
    delta = data.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Function to classify RSI signals based on interpretation
def classify_rsi(rsi):
    signals = []
    for val in rsi:
        if val > 70:
            signals.append("Overbought")
        elif val < 30:
            signals.append("Oversold")
        else:
            signals.append("Neutral")
    return signals

# Function to interpret RSI signals based on trend and divergence criteria
def interpret_rsi(data):
    data['RSI_14'] = calculate_rsi(data['Close'], 14)
    data['RSI_30'] = calculate_rsi(data['Close'], 30)
    data['RSI_14_Class'] = classify_rsi(data['RSI_14'])
    data['RSI_30_Class'] = classify_rsi(data['RSI_30'])
    # Trend interpretation
    data['Trend_14'] = np.where(data['RSI_14'].diff() > 0, "Rising", "Falling")
    data['Trend_30'] = np.where(data['RSI_30'].diff() > 0, "Rising", "Falling")
    # Divergence interpretation (for 14-day period)
    data['RSI_14_MA'] = data['RSI_14'].rolling(window=14).mean()
    data['Close_14_MA'] = data['Close'].rolling(window=14).mean()
    data['Bullish_Divergence'] = np.where((data['RSI_14'] > data['RSI_14_MA'].shift(1)) & (data['Close'] < data['Close_14_MA'].shift(1)), "Yes", "No")
    data['Bearish_Divergence'] = np.where((data['RSI_14'] < data['RSI_14_MA'].shift(1)) & (data['Close'] > data['Close_14_MA'].shift(1)), "Yes", "No")
    return data  # Return the modified DataFrame

# Read data from CSV files in the raw data path
for file_name in os.listdir(raw_data_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(raw_data_path, file_name)
        # Read CSV file
        df = pd.read_csv(file_path)
        # Interpret RSI signals
        df_interpreted = interpret_rsi(df)
        # Save interpreted RSI data to CSV file in the output path
        output_file_path = os.path.join(rsi_output_path, file_name)
        df_interpreted.to_csv(output_file_path, index=False, decimal=',')

print("RSI calculation and interpretation completed. Results saved to the RSI output path.")

RSI calculation and interpretation completed. Results saved to the RSI output path.


**RSI Data Consolidation**

In [6]:
# 05SUMRSI

import os
import pandas as pd

# Define paths
rsi_input_path = '/content/drive/MyDrive/DATA/RSI'
output_file_path = '/content/drive/MyDrive/DATA/results_rsi.csv'

# Function to read CSV files and extract rows with the latest date
def extract_latest_data(file_path, file_name):
    df = pd.read_csv(file_path)
    latest_date = df['Date'].max()  # Find the latest date in the dataframe
    latest_data = df[df['Date'] == latest_date].copy()  # Extract rows with the latest date and create a copy
    latest_data['File'] = file_name  # Add a new column with the file name
    return latest_data

# Initialize an empty list to store dataframes
dfs = []

# Iterate through CSV files in the RSI input path
for file_name in os.listdir(rsi_input_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(rsi_input_path, file_name)
        # Extract rows with the latest date from the current file
        latest_data = extract_latest_data(file_path, file_name)
        # Append the latest data to the list of dataframes
        dfs.append(latest_data)

# Concatenate dataframes in the list into a single dataframe
consolidated_data = pd.concat(dfs, ignore_index=True)

# Check if the output file already exists
if os.path.exists(output_file_path):
    # Remove the existing file
    os.remove(output_file_path)

# Save the consolidated data to a CSV file
consolidated_data.to_csv(output_file_path, index=False, decimal=',')

print("Latest RSI data with file names has been consolidated and saved to:", output_file_path)


Latest RSI data with file names has been consolidated and saved to: /content/drive/MyDrive/DATA/results_rsi.csv


##Calculating Moving average convergence and divergence (MACD) Indocator

**Usage**
- Dependencies: Ensure you have the required Python libraries installed, including pandas and numpy.

- File Paths: Define the paths for the raw data directory and the MACD output directory. Modify the raw_data_path and macd_output_path variables to match your directory structure.

- Data Preparation: Ensure your raw data CSV files are placed in the specified raw data directory. The script expects the files to have a 'Close' column for the calculation of MACD.

- Running the Script: Execute the script to calculate and interpret the Moving Average Convergence Divergence (MACD) for each CSV file in the raw data directory and save the results to the MACD output directory.

**Script Explanation**
The script performs the following key tasks:

- File Paths: Specifies the paths for the raw data directory and the directory where the MACD data will be saved.

- MACD Calculation: Defines a function to calculate the MACD parameters, including:

1. *MACD Line: The difference between the 12-day Exponential Moving Average (EMA) and the 26-day EMA.*
2. *MACD Signal Line: The 9-day EMA of the MACD line.*
3. *MACD Histogram: The difference between the MACD line and the MACD Signal line.*

- MACD Signal Interpretation: Defines a function to interpret MACD signals, including:

1. *Buy Signal: When the MACD crosses its Signal line from bottom to top.*
2. *Sell Signal: When the MACD crosses its Signal line from top to bottom.*
3. *Signal Intensity: Average distance of MACD and Signal line to the midpoint line.*
4. *Upward Momentum: When the MACD is higher than its nine-day average.*
5. *Downward Momentum: When the MACD is lower than its nine-day average.*
6. *Open Bullish Position: When the MACD shows an uptrend over a period of 14 days.*

- Open Bearish Position: When the MACD shows a downtrend over a period of 14 days.
- Remove Existing Files: Removes any existing CSV files in the MACD output directory to ensure fresh data storage.

- Read and Process Data: Iterates over all CSV files in the raw data directory, reads each file into a DataFrame, calculates the MACD parameters, interprets MACD signals, and saves the interpreted MACD data for each file to the specified MACD output directory.

- Execution: After processing all CSV files, the script prints a completion message indicating that the MACD calculation and interpretation are finished and the results are saved.

In [7]:
# 06MACD

import os
import pandas as pd
import numpy as np

# Define paths
raw_data_path = '/content/drive/MyDrive/DATA/RAW'
macd_output_path = '/content/drive/MyDrive/DATA/MACD'

# Function to calculate MACD parameters
def calculate_macd(data):
    # Calculate MACD line (12-day EMA - 26-day EMA)
    data['EMA_12'] = data['Close'].ewm(span=12, min_periods=0, adjust=False).mean()
    data['EMA_26'] = data['Close'].ewm(span=26, min_periods=0, adjust=False).mean()
    data['MACD'] = data['EMA_12'] - data['EMA_26']

    # Calculate MACD Signal line (9-day EMA of MACD)
    data['MACD_Signal'] = data['MACD'].ewm(span=9, min_periods=0, adjust=False).mean()

    # Calculate MACD Histogram (MACD - Signal)
    data['MACD_Histogram'] = data['MACD'] - data['MACD_Signal']

    return data

# Function to interpret MACD signals
def interpret_macd(data):
    # Buy Signal: MACD crosses its trigger from bottom to top
    data['Buy_Signal'] = np.where((data['MACD'] > data['MACD_Signal']) & (data['MACD'].shift(1) < data['MACD_Signal'].shift(1)), "Buy", "")

    # Sell Signal: MACD crosses its trigger from top to bottom
    data['Sell_Signal'] = np.where((data['MACD'] < data['MACD_Signal']) & (data['MACD'].shift(1) > data['MACD_Signal'].shift(1)), "Sell", "")

    # Signal Intensity: Average distance of MACD and Trigger line to the midpoint line
    data['Signal_Intensity'] = (data['MACD'] + data['MACD_Signal']) / 2

    # Upward Momentum: MACD is higher than its nine-day average
    data['Upward_Momentum'] = np.where(data['MACD'] > data['MACD'].rolling(window=9).mean(), "Upward Momentum", "")

    # Downward Momentum: MACD is lower than its nine-day average
    data['Downward_Momentum'] = np.where(data['MACD'] < data['MACD'].rolling(window=9).mean(), "Downward Momentum", "")

    # Open Bullish Position: MACD shows an uptrend over a period of 14 days
    data['Open_Bullish_Position'] = np.where(data['MACD'] > data['MACD'].shift(14), "Open Bullish Position", "")

    # Open Bearish Position: MACD shows a downtrend over a period of 14 days
    data['Open_Bearish_Position'] = np.where(data['MACD'] < data['MACD'].shift(14), "Open Bearish Position", "")

    return data

# Remove existing files in the output path
for file_name in os.listdir(macd_output_path):
    file_path = os.path.join(macd_output_path, file_name)
    os.remove(file_path)

# Read data from CSV files in the raw data path
for file_name in os.listdir(raw_data_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(raw_data_path, file_name)
        # Read CSV file
        df = pd.read_csv(file_path)
        # Calculate MACD parameters
        df = calculate_macd(df)
        # Interpret MACD signals
        df = interpret_macd(df)
        # Save interpreted MACD data to CSV file in the output path
        output_file_path = os.path.join(macd_output_path, file_name)
        df.to_csv(output_file_path, index=False, decimal=',')

print("MACD calculation and interpretation completed. Results saved to the MACD output path.")

MACD calculation and interpretation completed. Results saved to the MACD output path.


**MACD Data Consolidation**

In [8]:
# 07SUMMACD

import os
import pandas as pd

# Define paths
macd_input_path = '/content/drive/MyDrive/DATA/MACD'
output_file_path = '/content/drive/MyDrive/DATA/results_macd.csv'

# Function to read CSV files and extract rows with the latest date
def extract_latest_data(file_path, file_name):
    try:
        df = pd.read_csv(file_path)
        latest_date = df['Date'].max()  # Find the latest date in the dataframe
        latest_data = df[df['Date'] == latest_date].copy()  # Extract rows with the latest date and create a copy
        latest_data['File'] = file_name  # Add a new column with the file name
        return latest_data
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None

# Initialize an empty list to store dataframes
dfs = []

# Iterate through CSV files in the MACD input path
for file_name in os.listdir(macd_input_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(macd_input_path, file_name)
        # Extract rows with the latest date from the current file
        latest_data = extract_latest_data(file_path, file_name)
        if latest_data is not None:
            # Append the latest data to the list of dataframes
            dfs.append(latest_data)

# Check if any dataframes were extracted
if dfs:
    # Concatenate dataframes in the list into a single dataframe
    consolidated_data = pd.concat(dfs, ignore_index=True)

    # Check if the output file already exists and remove it
    if os.path.exists(output_file_path):
        os.remove(output_file_path)

    # Save the consolidated data to a CSV file
    consolidated_data.to_csv(output_file_path, index=False, decimal=',')
    print("Latest MACD data with file names has been consolidated and saved to:", output_file_path)
else:
    print("No MACD data found in the specified directory.")

Latest MACD data with file names has been consolidated and saved to: /content/drive/MyDrive/DATA/results_macd.csv


##Calculating Moving Averages
**Usage**
- Dependencies: Ensure you have the required Python libraries installed, including pandas.

- File Paths: Define the paths for the raw data directory and the output directory for the simple moving average (SMA) analysis. Adjust the raw_data_path and output_path variables to match your directory structure.

- Data Preparation: Place your raw data CSV files in the specified raw data directory. Each file should contain a 'Close' column for the calculation of SMAs.

- Running the Script: Execute the script to analyze each CSV file in the raw data directory using SMA indicators and save the results to the output directory.

**Script Explanation**
The script performs the following tasks:

- File Paths: Specifies the paths for the raw data directory and the directory where the SMA analysis results will be saved.

- Simple Moving Average (SMA) Calculation: Defines a function to calculate SMAs with different window lengths (e.g., 20-day, 50-day, 100-day, 200-day) for each CSV file:

- Calculates SMAs using the rolling mean function on the 'Close' price.
Interpretation of SMA Signals: Defines a function to interpret SMA signals based on two common technical analysis patterns:

1. *Golden Cross: When the shorter-term SMA crosses above the longer-term SMA, suggesting a potential buy signal.*
2. *Death Cross: When the shorter-term SMA crosses below the longer-term SMA, suggesting a potential sell signal.*
3. *Determines the interpretation based on changes in SMA values over time.*
- Processing Each CSV File: Iterates over all CSV files in the raw data directory:

1. *Reads each CSV file into a DataFrame.*
2. *Calculates multiple SMAs (e.g., 20-day vs 50-day, 100-day vs 200-day).*
3. *Interprets SMA signals using the defined functions.*
4. *Saves the processed DataFrame with SMA indicators and interpretation signals to a new CSV file in the output directory.*

- Execution: After processing all CSV files, the script prints a completion message indicating that the SMA analysis with modified interpretation is finished, and the results are saved in the specified output path.

In [9]:
# 08MOVAVG

import os
import pandas as pd
from IPython.display import Javascript

# Define paths
raw_data_path = '/content/drive/MyDrive/DATA/RAW'
output_path = '/content/drive/MyDrive/DATA/AVG'

# Function to calculate simple moving averages
def calculate_sma(data, short_window, long_window):
    short_ma = data['Close'].rolling(window=short_window).mean()
    long_ma = data['Close'].rolling(window=long_window).mean()
    return short_ma, long_ma

# Function to interpret SMA signals based on Golden Cross and Death Cross
def interpret_sma(short_ma, long_ma):
    interpretation = []
    for i in range(len(short_ma)):
        if short_ma[i] > long_ma[i] and short_ma[i - 1] <= long_ma[i - 1] and short_ma[i] > short_ma[i - 1] and long_ma[i] > long_ma[i - 1]:
            interpretation.append("Buy Signal - Golden Cross")
        elif short_ma[i] < long_ma[i] and short_ma[i - 1] >= long_ma[i - 1] and short_ma[i] < short_ma[i - 1] and long_ma[i] < long_ma[i - 1]:
            interpretation.append("Sell Signal - Death Cross")
        else:
            interpretation.append("No Signal")
    return interpretation

# Function to process each CSV file
def process_file(file_path):
    try:
        # Read CSV file
        df = pd.read_csv(file_path)
        # Calculate SMAs
        df['SMA_20'], df['SMA_50'] = calculate_sma(df, 20, 50)
        df['SMA_100'], df['SMA_200'] = calculate_sma(df, 100, 200)
        # Interpret SMA signals
        df['20_50_Signal'] = interpret_sma(df['SMA_20'], df['SMA_50'])
        df['100_200_Signal'] = interpret_sma(df['SMA_100'], df['SMA_200'])
        return df
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None

# Process each CSV file in the raw data path
for file_name in os.listdir(raw_data_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(raw_data_path, file_name)
        # Process file
        processed_df = process_file(file_path)
        if processed_df is not None:
            # Save processed data to CSV file in the output path
            output_file_path = os.path.join(output_path, file_name)
            processed_df.to_csv(output_file_path, index=False, decimal=',')

print("Simple Moving Average (SMA) analysis with modified interpretation completed.")

Simple Moving Average (SMA) analysis with modified interpretation completed.


**Moving Averge Data Consolidation**

In [10]:
# 09SUMMOVAVG

import os
import pandas as pd

# Define paths
avg_input_path = '/content/drive/MyDrive/DATA/AVG'
output_file_path = '/content/drive/MyDrive/DATA/results_avg.csv'

# Function to read CSV files and extract rows with the latest date
def extract_latest_data(file_path, file_name):
    try:
        df = pd.read_csv(file_path)
        latest_date = df['Date'].max()  # Find the latest date in the dataframe
        latest_data = df[df['Date'] == latest_date].copy()  # Extract rows with the latest date and create a copy
        latest_data['File'] = file_name  # Add a new column with the file name
        return latest_data
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None

# Initialize an empty list to store dataframes
dfs = []

# Iterate through CSV files in the AVG input path
for file_name in os.listdir(avg_input_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(avg_input_path, file_name)
        # Extract rows with the latest date from the current file
        latest_data = extract_latest_data(file_path, file_name)
        if latest_data is not None:
            # Append the latest data to the list of dataframes
            dfs.append(latest_data)

# Check if any dataframes were extracted
if dfs:
    # Concatenate dataframes in the list into a single dataframe
    consolidated_data = pd.concat(dfs, ignore_index=True)

    # Check if the output file already exists and remove it
    if os.path.exists(output_file_path):
        os.remove(output_file_path)

    # Save the consolidated data to a CSV file
    consolidated_data.to_csv(output_file_path, index=False, decimal=',')
    print("Latest average data with file names has been consolidated and saved to:", output_file_path)
else:
    print("No average data found in the specified directory.")

Latest average data with file names has been consolidated and saved to: /content/drive/MyDrive/DATA/results_avg.csv


# Data Consolidation
**Usage**
- Dependencies: Ensure you have the pandas library installed to run this script.

- Data Files: Prepare the input data files (results_stochastic.csv, results_rsi.csv, results_macd.csv, results_avg.csv) in the specified data directory (/content/drive/MyDrive/DATA/). These files should contain the results from individual technical analysis scripts.

- Output File: Define the output_file_path where the merged summary data will be saved as a CSV file.

- Running the Script: Execute the script to merge data from multiple CSV files based on a common column ('File') and save the summarized results to the specified output file.

**Script Explanation**
The script performs the following tasks:

- File Paths: Specifies the paths for the data directory containing individual technical analysis results (data_path) and the path to save the merged summary data (output_file_path).

- Reading Data: Reads each of the four CSV files (results_stochastic.csv, results_rsi.csv, results_macd.csv, results_avg.csv) into separate DataFrames (stochastic_data, rsi_data, macd_data, avg_data).

- Merging Dataframes: Combines the DataFrames based on a common column ('File'):

1. *First merges stochastic_data with rsi_data to include columns related to RSI indicators.*
2. *Then merges the resulting DataFrame with macd_data to incorporate MACD indicators.*
3. *Finally, merges with avg_data to add SMA signals.*

- Column Order: Adjusts the order of columns in the merged DataFrame, moving the 'File' column to the first position for clarity.

- Saving Data: Saves the merged DataFrame (merged_data) to a CSV file (output_file_path) without including the index and using ',' as the decimal separator.

- Completion Message: Prints a message confirming that the summary data has been saved to the specified output file.

This script enables consolidation and comparison of technical analysis results from multiple indicators into a single summary file for further analysis or reporting purposes. Adjustments can be made as needed to handle different sets of data or additional analysis criteria.

In [11]:
# 10MAIN

import pandas as pd

# Define file paths
data_path = '/content/drive/MyDrive/DATA/'
output_file_path = '/content/drive/MyDrive/DATA/results_main.csv'

# Read data from each file
stochastic_data = pd.read_csv(data_path + 'results_stochastic.csv')
rsi_data = pd.read_csv(data_path + 'results_rsi.csv')
macd_data = pd.read_csv(data_path + 'results_macd.csv')
avg_data = pd.read_csv(data_path + 'results_avg.csv')

# Merge dataframes on the 'File' column
merged_data = stochastic_data.merge(rsi_data[['File', 'RSI_14', 'RSI_30', 'RSI_14_Class', 'RSI_30_Class', 'Trend_14', 'Trend_30', 'Bullish_Divergence', 'Bearish_Divergence']], on='File', how='left')
merged_data = merged_data.merge(macd_data[['File', 'Buy_Signal', 'Sell_Signal', 'Signal_Intensity', 'Upward_Momentum', 'Downward_Momentum', 'Open_Bullish_Position', 'Open_Bearish_Position']], on='File', how='left')
merged_data = merged_data.merge(avg_data[['File', '20_50_Signal', '100_200_Signal']], on='File', how='left')

# Move the 'File' column to the first position
merged_data = merged_data[['File'] + [col for col in merged_data.columns if col != 'File']]

# Save the merged data to a CSV file
merged_data.to_csv(output_file_path, index=False, decimal=',')

print("Summary data saved to:", output_file_path)


Summary data saved to: /content/drive/MyDrive/DATA/results_main.csv
